In [ ]:
import pandas as pd
import os
from statsmodels.regression.linear_model import OLS
pd.options.mode.chained_assignment = None


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Read estimation data set into RAM
df = pd.read_csv('https://raw.githubusercontent.com/lazhizhin/dcml_2020/master/DC/sem_1/data/cereal_demogr_merged_processed.csv')
df = df.join(pd.get_dummies(df['city']))
df.drop(1, axis = 1, inplace = True)


In [ ]:
# Let's proceed to actual estimation of two-level demand
firmbrands = list(df['firmbr'].drop_duplicates())
#print(firmbrands)
firmssegments = df[['firmbr','segment']].drop_duplicates()
#print(firmssegments)
df.head()


In [ ]:
# Estimation here! #
## Preparation
# Define list of all unique firmbr, that are in df
firmbrands = list(df['firmbr'].drop_duplicates())
# Create table of firmbr-segment correspondence
firmssegments = df[['firmbr','segment']].drop_duplicates()
# Set independent variables set
vars_of_interest_rest = ['Age','Child','price_index','segment_income']
# cities_fe = df['city'].drop_duplicates()[1:]
# vars_of_interest_rest += list(cities_fe)
## Estimation
# Iterate over segments
for seg in range(1,5):
    # For each segment, define list of brands within given segment
    fimbrands_tmp = firmssegments[firmssegments['segment']==seg]
    # On this stage, we want only intra-segment elasticities, so
    # we select only prices for brands within given segment
    vars_of_interest_prices = list(fimbrands_tmp['firmbr'].astype(str))
    # Iterate over firmbrands
    for fb in fimbrands_tmp['firmbr']:
        # Create list of variables to use
        vars_of_interest = vars_of_interest_prices + vars_of_interest_rest
        # Use only shares of given brand
        df_tmp = df[df['firmbr']==fb]
        # Define target variable
        Y = df_tmp['share']
        # Define matrix of independent variables
        X = df_tmp[vars_of_interest]
        X.loc[:,'Intercept'] = 1
        # Create OLS model object
        model = OLS(Y,X)
        # Fit the model
        results = model.fit()
        # Save estimation results
        regtable = results.summary2().tables[1]
        # Produce easy-to-use point estimates
        regtable['Coef.'] = regtable['Coef.'].round(3).astype(str)
        regtable.loc[regtable['P>|t|']<0.1,'Coef.'] += '*'
        regtable.loc[regtable['P>|t|']<0.05,'Coef.'] += '*'
        regtable.loc[regtable['P>|t|']<0.01,'Coef.'] += '*'
        # Keep only point estimates
        regtable = regtable[['Coef.']]
        regtable.columns = [str(fb)]
        # If coefficient matrix for given segment exists, join current results
        try:
            globals()['table_out'+str(seg)] = globals()['table_out'+str(seg)].join(regtable)
        # If doesn't exist -- create one
        except:
            globals()['table_out'+str(seg)] = regtable.copy()

In [ ]:
#table_out4

,1009,1030,2005,2048,3014,4010,6018
1009,-0.34***,0.051,0.094,0.093**,-0.013,0.048,0.028
1030,-0.012,-0.192***,0.108,0.053,-0.121,-0.012,-0.031
2005,-0.018,-0.031,-0.563***,-0.001,0.17,-0.09,-0.095
2048,-0.017,0.006,0.017,-0.244***,-0.149,-0.014,0.111
3014,0.015,0.009,-0.004,0.006,-0.819***,0.029,0.049
4010,-0.075*,-0.034,0.015,-0.019,-0.031,-0.227***,-0.038
6018,-0.113**,0.062,-0.14,0.051,0.089,0.015,-0.583***
Age,-0.002,0.001,-0.012,-0.007*,0.027*,0.005,-0.013
Child,-0.002,0.002,-0.006,0.002,-0.027**,0.006,-0.004
price_index,0.004,0.153**,0.537**,0.12*,0.447**,0.0,0.408***


In [ ]:
#table_out1

,1006,2028,3006
1006,-0.812***,-0.152,-0.027
2028,0.202,-0.491***,0.005
3006,0.365**,0.057,-0.346***
Age,-0.005,0.01,0.008
Child,0.012,0.013*,0.012*
price_index,0.801***,0.32**,0.044
segment_income,0.141,0.022,-0.049
Intercept,0.01,0.105***,0.061***


In [ ]:
df_segment_level = pd.read_csv('https://raw.githubusercontent.com/lazhizhin/dcml_2020/master/DC/sem_1/data/cereal_demogr_merged_processed_segment_lvl.csv')

In [ ]:
## Estimate segment-level OLS
df_tmp = df_segment_level[df_segment_level['segment'] == 1]
# 1. Define target variable Y
Y = df_tmp['share']
# 2. Define matrix of independent variables X
X = df_tmp[['cereal_exp','1.0','2.0','3.0','4.0']]
X['Intercept'] = 1
# 3. Create OLS model object
model = OLS(Y,X)
# 4. Fit model with .fit() method
results = model.fit()
# 5. Call results tables(see documentation for summary() and summary2())
regtable = results.summary()
regtable

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  share   R-squared:                       0.759
Model:                            OLS   Adj. R-squared:                  0.745
Method:                 Least Squares   F-statistic:                     55.44
Date:                Thu, 24 Dec 2020   Prob (F-statistic):           9.80e-26
Time:                        11:03:25   Log-Likelihood:                 219.74
No. Observations:                  94   AIC:                            -427.5
Df Residuals:                      88   BIC:                            -412.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cereal_exp     0.0160      0.024      0.665      0.508      -0.032       0.064
1.0            2.0794      0.388      5.361      0.000       1.308       2.850
2.0           -1.5174      0.443     -3.421      0.001      -2.399      -0.636
3.0           -1.6718      0.312     -5.360      0.000      -2.292      -1.052
4.0           -1.7504      0.410     -4.271      0.000      -2.565      -0.936
Intercept      0.1989      0.041      4.898      0.000       0.118       0.280
==============================================================================
Omnibus:                       16.266   Durbin-Watson:                   1.570
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               24.681
Skew:                           0.755   Prob(JB):                     4.37e-06
Kurtosis:                       5.006   Cond. No.                         282.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""